Data: 20/09/2024

In [33]:
import Orange
import numpy as np

### Base de Dados - Risco de Crédito

In [2]:
db_RiscoCredito = Orange.data.Table('risco_credito_regras.csv')

In [9]:
# O caractere C# separa o que é previsor e o que é classe
db_RiscoCredito

[[ruim, alta, nenhuma, 0_15 | alto],
 [desconhecida, alta, nenhuma, 15_35 | alto],
 [desconhecida, baixa, nenhuma, 15_35 | moderado],
 [desconhecida, baixa, nenhuma, acima_35 | alto],
 [desconhecida, baixa, nenhuma, acima_35 | baixo],
 ...
]

In [10]:
db_RiscoCredito.domain

[historia, divida, garantias, renda | risco]

In [11]:
# Algoritmo de regras de associação (CN2)
cn2 = Orange.classification.rules.CN2Learner()
# Criação das regras == Treinamento do algoritmo
regras_risco_credito = cn2(db_RiscoCredito)

In [16]:
# Resultado da Aprendizagem
for regra in regras_risco_credito.rule_list:
    print(regra)

IF renda==0_15 THEN risco=alto 
IF historia==boa AND divida!=alta THEN risco=baixo 
IF historia==boa AND garantias!=nenhuma THEN risco=baixo 
IF historia==boa AND renda!=15_35 THEN risco=baixo 
IF historia==boa THEN risco=moderado 
IF divida==alta THEN risco=alto 
IF historia!=desconhecida THEN risco=moderado 
IF garantias==adequada THEN risco=baixo 
IF renda==15_35 THEN risco=moderado 
IF historia==desconhecida THEN risco=baixo 
IF TRUE THEN risco=alto 


Simulação da Classificação de dois registros:  
- história boa, dívida alta, garantias nenhuma, renda > 35
- história ruim, dívida alta, garantias adequada, renda < 15

In [17]:
cliente1: list[str] = ['boa', 'alta', 'nenhuma', 'acima_35']
cliente2: list[str] = ['ruim', 'alta', 'adequada', '0_15']

clientes = [cliente1, cliente2]

previsoes = regras_risco_credito(clientes)

In [28]:
# Visualizar o resultado
print(f'registro pertecem as classes: {previsoes}')

# Representação da base em name e values
db_RiscoCredito.domain.class_var.name, db_RiscoCredito.domain.class_var.values

registro pertecem as classes: [1 0]


('risco', ('alto', 'baixo', 'moderado'))

In [32]:
for i in previsoes:
    # previsoes = [1, 0]
    print(db_RiscoCredito.domain.class_var.values[i])

baixo
alto


### Base de Dados - Credito

In [34]:
db_credit = Orange.data.Table('credit_data_regras.csv')

In [37]:
# divisao da base de dados. 'n' é qual % da base de TESTE, aqui 25%
base_teste, base_treinamento = Orange.evaluation.testing.sample(db_credit, n=0.25)

In [53]:
len(base_treinamento), len(base_teste)

(1500, 500)

In [39]:
# Criar/Treinar o algoritmo e treinar 
cn2 = Orange.classification.rules.CN2Learner()
regras_credit = cn2(base_treinamento)

In [40]:
for regra in regras_credit.rule_list:
    print(regra)

IF age>=34.851817262359 THEN default=0 
IF age>=34.7514377613552 THEN default=1 
IF loan<=2495.13299137587 AND income>=20511.4294355735 THEN default=0 
IF income<=31702.3342987522 AND age>=34.7410444347188 THEN default=0 
IF income<=31702.3342987522 AND age>=32.1632712800056 THEN default=1 
IF loan>=7708.31562497011 AND loan>=9698.582169129 THEN default=1 
IF age<=18.055874488782102 AND income>=50501.7266888171 THEN default=0 
IF loan>=8045.4409532236705 THEN default=1 
IF income<=45360.716274008206 AND loan>=5233.66322803221 THEN default=1 
IF age>=34.058644853998 THEN default=0 
IF income>=57596.3540980592 AND age>=22.939635145478 THEN default=0 
IF loan>=6536.96636294544 AND age>=22.3484194725752 THEN default=1 
IF age>=31.9685264034197 AND age>=33.804503521882 THEN default=1 
IF age>=32.718134177383 THEN default=0 
IF income<=31702.3342987522 AND age>=31.9685264034197 THEN default=0 
IF income<=31702.3342987522 AND loan>=3665.88089899456 THEN default=1 
IF age>=32.704374910332305 T

In [43]:
pred_credit = Orange.evaluation.testing.TestOnTestData(data=base_treinamento, test_data=base_teste, learners=[lambda testdata: regras_credit])

**Função Lambda no Contexto**: 

A função lambda `lambda testdata: regras_credit` basicamente serve como um "learner" personalizado, onde, em vez de treinar um novo modelo nos dados de treinamento, você já tem o `modelo (regras_credit)` e apenas o aplica diretamente aos dados de teste.

`testdata`: Representa os `dados de teste`, e a função lambda está especificando que, quando os *dados de teste são passados*, eles devem ser *processados pelas regras* (`regras_credit`).

In [55]:
print(type(pred_credit))

<class 'Orange.evaluation.testing.Results'>


In [54]:
Orange.evaluation.CA(pred_credit)

array([0.99])